In [26]:
import pandas as pd
from datetime import datetime

rawpath = "../raw/"

In [ ]:
{'signatur':'string','jahr':'string','datum':'string','medienart':'string','sprache':'string','land':'string','koerperschaft_1':'string','koerperschaft_2':'string','person':'string', 'titel':'string','urheber_vorlage':'string','ort':'string','ort_idn':'string','umfang':'string','bemerkung':'string'}

In [14]:
df = pd.read_excel(f"{rawpath}/2022-ES-Tabelle.xlsx", names=['signatur','jahr','datum','medienart','sprache','land','koerperschaft_1','koerperschaft_2','person', 'titel','urheber_vorlage','ort','ort_idn','umfang','bemerkung'], dtype={'signatur':'string','jahr':'string','datum':'string','medienart':'string','sprache':'string','land':'string','koerperschaft_1':'string','koerperschaft_2':'string','person':'string', 'titel':'string','urheber_vorlage':'string','ort':'string','ort_idn':'string','umfang':'string','bemerkung':'string'})
df = df.apply(lambda x: x.str.strip())
df

/home/andre/.cache/pypoetry/virtualenvs/base-_zdkNckr-py3.10/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,signatur,jahr,datum,medienart,sprache,land,koerperschaft_1,koerperschaft_2,person,titel,urheber_vorlage,ort,ort_idn,umfang,bemerkung
0,Bö-GR/V/1,1870,02.01.1870,druck,ger,XA-DXDE,Franz Vahlen,<NA>,<NA>,Begründung durch Ankauf von Artikeln des Verlages von G. Grote (C. Müller) und I. Guttentag.,Franz Vahlen,Berlin,!040057283!,5 Exemplare,Verlagsliste
1,Bö-GR/V/2,1886,XX.10.1886,druck,ger,XA-DXDE,Franz Vahlen,<NA>,<NA>,Vollmachtserteilung an den Mitarbeiter Friedrich Gebhardt.,Franz Vahlen,Berlin W.,!040057283!,6 Exemplare,<NA>
2,Bö-GR/V/3,1898,07.02.1898,druck,ger,XA-DXDE,Franz Vahlen,<NA>,<NA>,Der Mitarbeiter Friedrich Gebhardt wird Teilhaber.,Franz Vahlen,Berlin W.,!040057283!,4 Exemplare,<NA>
3,Bö-GR/V/4,1898,05.11.1898,druck,ger,XA-DXDE,Franz Vahlen,<NA>,"Gehardt, Friedrich",Friedrich Gebhardt kauft die Verlagsbuchhandlung.,Friedrich Gebhardt,Berlin W.8,!040057283!,6 Exemplare,Handschriftliches Begleitschreiben an den Börsenverein.
4,Bö-GR/V/5,1910,01.01.1910,druck,ger,XA-DXDE,Franz Vahlen,<NA>,<NA>,Der Mitarbeiter Heinrich Heise erhält Prokura.,Franz Vahlen,Berlin W 9,!040057283!,1 Exemplar,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,Bö-GR/V/78,1888,01.04.1888,druck,ger,XA-DXDE,Gottfried Veith,<NA>,<NA>,Verkauf an Gisbert Pillmeyer aus Hannover. N.F.: G. Pillmeyer (vorm. Gottfr. Veith's Buchhandlung).,Gottfr. Veith,Osnabrück,!040439747!,5 Exemplare,Zeugnisse. Handschriftliches Begleitschreiben von Pillmeyer an J. U. Kern in Breslau.
78,Bö-GR/V/79,1843,15.08.1843,druck,ger,XA-CH,J. Veith,<NA>,<NA>,Errichtung eines artistischen Verlages.,J. Veith,Zürich,!04068038X!,1 Exemplar,"Verlagswerke. Beiliegend: Anzeige für ""Almanach von Radierungen"" von M. von Schwind."
79,Bö-GR/V/80,1838,XX.06.1838,druck,ger,XA-DXDE,Veith & Rieger'sche Buchhandlung,<NA>,"Veith, Martin",Verkauf an Joh. Georg Lampart und Ed. V. Jenisch. N.F.: Lampart & Comp. (vormals Veith & Rieger'sche Buchhandlung).,"Martin Veith, Veith & Rieger'sche Buchhandlung",Augsburg,!040036146!,2 Exemplare,Zeugnisse. Verlagsliste.
80,Bö-GR/V/81,1828,25.01.1828,druck,ger,XA-DXDE,Mart. Veith- und Mich. Rieger'sche Buchhandlung,<NA>,"Veith, Martin Johann Baptist",Verkauf des Sortiments an Wilhelm von Manstein aus Wien. N.F.: Wilhelm von Manstein.,Martin Johann Baptist Veith. Firma: Mart. Veith- und Mich. Rieger'sche Buchhandlung,Augsburg,!040036146!,1 Exemplar,<NA>


In [28]:
def pica_schreiben(row):
    pica = f"""0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 {row.jahr}
1110 {row.datum}$4ezth
1130 !040445224!
1131 !040082407!
1132 a1-analog;a2-{row.medienart};f1-text;f2-blatt
1500 /1{row.sprache}
1700 /1{row.land}
3100 {row.koerperschaft_1}$BVerfasser$4aut
"""
    if pd.notna(row.koerperschaft_2):
        pica += f"3110 {row.koerperschaft_2}$BVerfasser$4aut\n"
    
    if pd.notna(row.person):
        pica += f"3010 {row.person}$BVerfasser$4aut\n"
    
    pica += f"""4000 {row.titel} / {row.urheber_vorlage}
4019 Geschäftsrundschreiben$Bobja
4046 {row.ort}
4060 {row.umfang}
4105 !1059146037!
4105 !1077661932!
"""
    if pd.notna(row.bemerkung):
        pica += f"4222 {row.bemerkung}\n"
    
    pica += f"""5560 [RNA]Korrespondenz
5590 [Entstehungsort]
5590 {row.ort_idn}
5591 [Objektgattung]
5591 !042993512!
7001 {now.strftime('%d-%m-%y')} : x
4800 !009033645!
6710 !103243757X!
7100 {row.signatur} @ m
7109 !!DBSM/S!! ; {row.signatur}
\t\n"""
    return pica

In [29]:
now = datetime.now()
for index, row in df.sample(2).iterrows():
    print(pica_schreiben(row))

0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1837
1110 XX.01.1837$4ezth
1130 !040445224!
1131 !040082407!
1132 a1-analog;a2-druck;f1-text;f2-blatt
1500 /1ger
1700 /1XA-DXDE
3100 Franz Varrentrapp$BVerfasser$4aut
3010 Krebs, Jacob$BVerfasser$4aut
4000 J. Krebs übergibt das Geschäft seinem ältesten Sohn Philipp. / J. Krebs, Theilhaber der Andreäischen Buchhandlung dahier
4019 Geschäftsrundschreiben$Bobja
4046 Frankfurt am Main
4060 2 Exemplare
4105 !1059146037!
4105 !1077661932!
5560 [RNA]Korrespondenz
5590 [Entstehungsort]
5590 !040181189!
5591 [Objektgattung]
5591 !042993512!
7001 07-09-22 : x
4800 !009033645!
6710 !103243757X!
7100 Bö-GR/V/44 @ m
7109 !!DBSM/S!! ; Bö-GR/V/44
	

0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1869
1110 01.11.1869$4ezth
1130 !040445224!
1131 !040082407!
1132 a1-analog;a2-druck;f1-text;f2-blatt
1500 /1ger
1700 /1XA-DXDE
3100 J. Veith$BVerfasser$4aut
3010 Veith, Eduard$BVer

In [30]:
now = datetime.now()
with open(f"../dat/rundschreiben-{now.strftime('%y-%m-%d-%H-%M-%S')}.dat","w") as f:
    with open(f"../dat/rundschreiben-recent.dat", 'w') as f2:
        for index, row in df.iterrows():
            f.write(pica_schreiben(row))
            f2.write(pica_schreiben(row))

with open(f"../dat/rundschreiben-recent-sample.dat", 'w') as f:
    for index, row in df.sample(5).iterrows():
        f.write(pica_schreiben(row))